In [10]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
%matplotlib inline

In [3]:
df = pd.read_csv(os.path.join('data', 'final_clf.csv')).drop('Unnamed: 0', axis=1)
df.head()

,budget,id,revenue,runtime,views,likes,dislikes,n_trailers,daysSinceStart,dayInYear,...,hi,it,ja,ko,ml,ru,ta,zh,profit,isProfitable
0,30000000,862,373554033.0,81.0,8050136,6330,1965,3,29484,302,...,0,0,0,0,0,0,0,0,343554033.0,1
1,65000000,8844,262797249.0,104.0,83048,70,14,1,29530,348,...,0,0,0,0,0,0,0,0,197797249.0,1
2,16000000,31357,81452156.0,127.0,104948,0,0,1,29537,355,...,0,0,0,0,0,0,0,0,65452156.0,1
3,60000000,949,187436818.0,170.0,32419,119,1,1,29530,348,...,0,0,0,0,0,0,0,0,127436818.0,1
4,35000000,9091,64350171.0,106.0,6162,12,3,2,29537,355,...,0,0,0,0,0,0,0,0,29350171.0,1


In [4]:
X = df.drop(['id', 'profit', 'isProfitable'], axis=1)
y = df.isProfitable

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4962 entries, 0 to 4961
Data columns (total 44 columns):
budget             4962 non-null int64
revenue            4962 non-null float64
runtime            4962 non-null float64
views              4962 non-null int64
likes              4962 non-null int64
dislikes           4962 non-null int64
n_trailers         4962 non-null int64
daysSinceStart     4962 non-null int64
dayInYear          4962 non-null int64
Mystery            4962 non-null int64
Romance            4962 non-null int64
Family             4962 non-null int64
Science Fiction    4962 non-null int64
Horror             4962 non-null int64
Crime              4962 non-null int64
Drama              4962 non-null int64
Fantasy            4962 non-null int64
Western            4962 non-null int64
Animation          4962 non-null int64
Music              4962 non-null int64
Adventure          4962 non-null int64
Foreign            4962 non-null int64
Action             4962 non-nul

In [6]:
def baseline():
    # create model
    model = Sequential()
    model.add(Dense(44, input_dim=44, kernel_initializer='normal', activation='relu'))
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
callback = ModelCheckpoint('weights.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model = baseline()
model.fit(X, y, validation_split=0.25, epochs=100, batch_size=10, callbacks=[callback], verbose=1)

Train on 3721 samples, validate on 1241 samples
Epoch 1/100
3721/3721 [==============================] - 2s 622us/step - loss: 1.3431 - acc: 0.7192 - val_loss: 0.6298 - val_acc: 0.6962

Epoch 00001: val_acc improved from -inf to 0.69621, saving model to weights.hdf5
Epoch 2/100
3721/3721 [==============================] - 1s 334us/step - loss: 0.6028 - acc: 0.7194 - val_loss: 0.6140 - val_acc: 0.6962

Epoch 00002: val_acc did not improve
Epoch 3/100
3721/3721 [==============================] - 1s 335us/step - loss: 0.5941 - acc: 0.7194 - val_loss: 0.6152 - val_acc: 0.6962

Epoch 00003: val_acc did not improve
Epoch 4/100
3721/3721 [==============================] - 1s 332us/step - loss: 0.5937 - acc: 0.7194 - val_loss: 0.6153 - val_acc: 0.6962

Epoch 00004: val_acc did not improve
Epoch 5/100
3721/3721 [==============================] - 1s 333us/step - loss: 0.5936 - acc: 0.7194 - val_loss: 0.6146 - val_acc: 0.6962

Epoch 00005: val_acc did not improve
Epoch 6/100
3721/3721 [==========

3721/3721 [==============================] - 1s 333us/step - loss: 0.5937 - acc: 0.7194 - val_loss: 0.6154 - val_acc: 0.6962

Epoch 00047: val_acc did not improve
Epoch 48/100
3721/3721 [==============================] - 1s 333us/step - loss: 0.5938 - acc: 0.7194 - val_loss: 0.6154 - val_acc: 0.6962

Epoch 00048: val_acc did not improve
Epoch 49/100
3721/3721 [==============================] - 1s 332us/step - loss: 0.5938 - acc: 0.7194 - val_loss: 0.6159 - val_acc: 0.6962

Epoch 00049: val_acc did not improve
Epoch 50/100
3721/3721 [==============================] - 1s 335us/step - loss: 0.5936 - acc: 0.7194 - val_loss: 0.6161 - val_acc: 0.6962

Epoch 00050: val_acc did not improve
Epoch 51/100
3721/3721 [==============================] - 1s 339us/step - loss: 0.5937 - acc: 0.7194 - val_loss: 0.6158 - val_acc: 0.6962

Epoch 00051: val_acc did not improve
Epoch 52/100
3721/3721 [==============================] - 1s 340us/step - loss: 0.5935 - acc: 0.7194 - val_loss: 0.6149 - val_acc: 0.


Epoch 00093: val_acc did not improve
Epoch 94/100
3721/3721 [==============================] - 1s 334us/step - loss: 0.5937 - acc: 0.7194 - val_loss: 0.6156 - val_acc: 0.6962

Epoch 00094: val_acc did not improve
Epoch 95/100
3721/3721 [==============================] - 1s 332us/step - loss: 0.5937 - acc: 0.7194 - val_loss: 0.6153 - val_acc: 0.6962

Epoch 00095: val_acc did not improve
Epoch 96/100
3721/3721 [==============================] - 1s 337us/step - loss: 0.5936 - acc: 0.7194 - val_loss: 0.6153 - val_acc: 0.6962

Epoch 00096: val_acc did not improve
Epoch 97/100
3721/3721 [==============================] - 1s 336us/step - loss: 0.5937 - acc: 0.7194 - val_loss: 0.6151 - val_acc: 0.6962

Epoch 00097: val_acc did not improve
Epoch 98/100
3721/3721 [==============================] - 1s 334us/step - loss: 0.5937 - acc: 0.7194 - val_loss: 0.6152 - val_acc: 0.6962

Epoch 00098: val_acc did not improve
Epoch 99/100
3721/3721 [==============================] - 1s 336us/step - loss: 0.

In [8]:
roc_auc_score(y, model.predict_proba(X))

0.5

In [ ]:
results = cross_val_score(clf, X, y, cv=kfold)
print('Accuracy %.2f%%' % (results.mean()*100))

In [ ]:
roc_auc_score(y, clf.predict(X))

In [ ]:
def shrink_baseline():
    model = Sequential()
    model.add(Dense(22, input_dim=44, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
clf = KerasClassifier(build_fn=shrink_baseline, epochs=100, batch_size=10, verbose=1)

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = cross_val_score(clf, X, y, cv=kfold)
print('Accuracy %.2f%%' % (results.mean()*100))